In [ ]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pytz
import requests_cache
%matplotlib inline
import seaborn as sns

from pytus2000 import read_diary_file, diary, read_individual_file, individual, read_diary_file_as_timeseries
import pytus2000
import people as ppl
module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)
import ktp.tus

In [ ]:
BUILD_FOLDER_PATH = Path('./build/')
TUS_DATA_FOLDER_PATH = Path('./data/UKDA-4504-tab/')

In [ ]:
pytus2000.set_cache_location(BUILD_FOLDER_PATH)
requests_cache.install_cache((BUILD_FOLDER_PATH / 'web-cache').as_posix())

## Participants

In [ ]:
individual_data = read_individual_file(TUS_DATA_FOLDER_PATH / 'tab' / 'individual_data_5.tab')
## TODO filter city population
seed = pd.DataFrame(index=individual_data.index, columns=['labour', 'qualification', 'age', 'hhtype'])
seed.labour = individual_data.ECONACT2.map(ktp.tus.LABOUR_MAP)
seed.labour[individual_data.IAGE > 74] = ktp.census.Labour.ABOVE_74
seed.qualification = individual_data.HIQUAL4.map(ktp.tus.QUALIFICATION_MAP)
seed['age'] = individual_data.IAGE.copy()
seed['hhtype'] = individual_data.HHTYPE4.map(ktp.tus.HOUSEHOLDTYPE_MAP)
## TODO add more feature, all that are in census data as well
seed.head()

In [ ]:
seed.dropna(axis='index', how='any', inplace=True)
assert not seed.isnull().any().any()

In [ ]:
household_types = seed.groupby((seed.index.get_level_values(0), seed.index.get_level_values(1))).hhtype.first()
household_sizes = seed.groupby((seed.index.get_level_values(0), seed.index.get_level_values(1))).hhtype.count()
mask_couples_children = household_sizes[(household_types == ktp.census.HouseholdType.COUPLE_WITH_DEPENDENT_CHILDREN) & (household_sizes <= 2)]
mask_couples_no_children = household_sizes[(household_types == ktp.census.HouseholdType.COUPLE_WITHOUT_DEPENDENT_CHILDREN) & (household_sizes != 2)]
invalids = (
    household_sizes[(household_types == ktp.census.HouseholdType.COUPLE_WITH_DEPENDENT_CHILDREN) & (household_sizes <= 2)] |
    household_sizes[(household_types == ktp.census.HouseholdType.COUPLE_WITHOUT_DEPENDENT_CHILDREN) & (household_sizes != 2)] |
    household_sizes[(household_types == ktp.census.HouseholdType.LONE_PARENT_WITH_DEPENDENT_CHILDREN) & (household_sizes < 2)] |
    household_sizes[(household_types == ktp.census.HouseholdType.MULTI_PERSON_HOUSEHOLD) & (household_sizes <= 2)]
)

seed.drop(labels=invalids.index, level=None, inplace=True)


print("{} households are invalid and were removed.".format(invalids.count()))

### Test

In [ ]:
assert not ((seed.labour == ktp.census.Labour.ABOVE_74) & (seed.age <= 74)).any()
assert not ((seed.labour == ktp.census.Labour.BELOW_16) & (seed.age >= 16)).any()
assert not ((seed.labour == ktp.census.Labour.BELOW_16) & (seed.qualification != ktp.census.Qualification.BELOW_16)).any()
assert not ((seed.labour != ktp.census.Labour.BELOW_16) & (seed.qualification == ktp.census.Qualification.BELOW_16)).any()

household_types = seed.groupby((seed.index.get_level_values(0), seed.index.get_level_values(1))).hhtype.first()
household_sizes = seed.groupby((seed.index.get_level_values(0), seed.index.get_level_values(1))).hhtype.count()
assert (household_sizes[household_types == ktp.census.HouseholdType.ONE_PERSON_HOUSEHOLD] == 1).all()
assert (household_sizes[household_types == ktp.census.HouseholdType.COUPLE_WITH_DEPENDENT_CHILDREN] > 2).all()
assert (household_sizes[household_types == ktp.census.HouseholdType.LONE_PARENT_WITH_DEPENDENT_CHILDREN] > 1).all()
assert (household_sizes[household_types == ktp.census.HouseholdType.COUPLE_WITHOUT_DEPENDENT_CHILDREN] == 2).all()
assert (household_sizes[household_types == ktp.census.HouseholdType.MULTI_PERSON_HOUSEHOLD] > 2).all()

## Time Series

In [ ]:
diary_data = read_diary_file(TUS_DATA_FOLDER_PATH / 'tab' / 'diary_data_8.tab')
diary_data_ts = read_diary_file_as_timeseries(TUS_DATA_FOLDER_PATH / 'tab' / 'diary_data_8.tab')[['activity', 'location']]

In [ ]:
simple_ts = pd.DataFrame({
    'location': diary_data_ts.location.map(ktp.tus.LOCATION_MAP),
    'activity': diary_data_ts.activity.map(ktp.tus.ACTIVITY_MAP)
})

### Handle Unknowns

In [ ]:
simple_ts.isnull().any()

There are no nans.

In [ ]:
len(simple_ts[(simple_ts.activity == ktp.tus.Activity.UNKNOWN) | (simple_ts.location == ktp.tus.Location.UNKNOWN)]) / len(simple_ts)

5.5% of all entries are missing.

In [ ]:
filled_simple_ts = simple_ts.copy()

In [ ]:
filled_simple_ts.replace(to_replace=[ktp.tus.Location.UNKNOWN, ktp.tus.Activity.UNKNOWN], value=np.nan, inplace=True)

In [ ]:
filled_simple_ts.isnull().describe()

Unknowns will be filled by forward fill. That is, whenever  an acticity/location is unknown it is expected that the last known activity/location is still valid. 

When doing that, it is important to not forward fill between diaries (all diaries are below each other). Hence, they must be grouped into diaries first and then forward filled. This will lead to the fact that not all Unknowns can be filled (the ones at the beginning of the day), but that is wanted.  

In [ ]:
filled_simple_ts = filled_simple_ts.groupby([filled_simple_ts.index.get_level_values(0), 
                                             filled_simple_ts.index.get_level_values(1), 
                                             filled_simple_ts.index.get_level_values(2), 
                                             filled_simple_ts.index.get_level_values(3)]).fillna(method='ffill')

In [ ]:
filled_simple_ts.isnull().describe()

In [ ]:
# TODO don't forward fill over too long durations, e.g. not more than 1-2h.

The remaining nans are filtered in the Filter section below.

### Map to markov states.

In [ ]:
markov_ts = ktp.tus.from_simplified_location_and_activity_to_people_model(filled_simple_ts)

### Filter

In [ ]:
# TODO

In [ ]:
def filter_nan(markov_ts, diary_data):
    """Remove all diaries with at least one NaN."""
    nan_mask = markov_ts.groupby(by=lambda index: (index[0], index[1], index[2], index[3])).apply(lambda values: values.isnull().any())
    return pd.DataFrame(markov_ts)[markov_ts.index.droplevel(4).isin(nan_mask[~nan_mask].index)]

def remove_people_not_in_seed(markov_ts, seed):
    mask = markov_ts.index.droplevel([3, 4]).isin(seed.index)
    return markov_ts[mask]


def filter_all_people_for_which_less_than_two_diaries_exist(markov_ts):
    valid_mask = markov_ts.groupby([markov_ts.index.get_level_values(0), 
                                    markov_ts.index.get_level_values(1), 
                                    markov_ts.index.get_level_values(2)]).apply(lambda values: len(values) == 24 * 6 * 2)
    return markov_ts[markov_ts.index.droplevel([3, 4]).isin(valid_mask[valid_mask].index)]

In [ ]:
markov_ts = filter_nan(markov_ts, diary_data)
assert not markov_ts.isnull().any().any()

In [ ]:
markov_ts = remove_people_not_in_seed(markov_ts, seed)
markov_ts = filter_all_people_for_which_less_than_two_diaries_exist(markov_ts)
seed = seed[seed.index.isin(markov_ts.index.droplevel([3, 4]))]
assert len(seed.index) * 2 * 24 * 6 == len(markov_ts.index)

In [ ]:
## TODO exchange SN4 with day type

### Add DayType

In [ ]:
weekdays = diary_data[diary_data.DDAYW2 == diary.DDAYW2.WEEKDAY_MON___FRI]
markov_ts['daytype'] = 'weekend'
markov_ts.loc[markov_ts.index.droplevel(4).isin(weekdays.index), 'daytype'] = 'weekday'
markov_ts = markov_ts.reset_index(level=[3, 4]).set_index(['daytype', 'time_of_day'], append=True)
markov_ts.drop('SN4', axis=1, inplace=True)
markov_ts.head()

In [ ]:
markov_ts = markov_ts.unstack([0, 1, 2])

## Select Variables

In [ ]:
markov_ts.ix['weekend', 10].map(lambda x: x.value).plot()
_ = plt.yticks([1, 2, 3, 4, 5], [x for x in ppl.Activity])